## 1st Order Non Linear ODEs

In [1]:
using LinearAlgebra
# using Plots
using PyCall
using PyPlot
using DataFrames
using CSV
using Symbolics
using Integrals
# plotlyjs()
matplotlib.use("TkAgg")

### ODE Algorithms
Here the RK4 algorithm is implemented as the following: <br>
$y_{n+1} = y_n + \frac{1}{6}(k_1+2k_2+2k_3+k_4)$ where:
1. $k_1 = hf(x_n,y_n)$
2. $k_2 = hf(x_n+\frac{1}{2}h,y_n+\frac{1}{2}k_1)$
3. $k_3 = hf(x_n+\frac{1}{2}h,y_n+\frac{1}{2}k_2)$
4. $k_2 = hf(x_n+h,y_n+k_3)$

The Adams Multon algorithms are also implemented as: <br>
1. Predictor: $y_{n+1} = y_n + \frac{1}{24}(55f_k - 59f_{k-1} + 37f_{k-2}-9f_{k-4})$
2. Predictor: $y_{n+1} = y_n + \frac{1}{24}(9f_{k+1} +19f_{k} - 5f_{k-1}+f_{k-2})$

In [2]:
function RK4(f::Function,xlim,y₀,h)
    xᵢ = xlim[1]:h:xlim[2]
    yᵢ = [y₀;zeros(length(xᵢ)-1)]
    
    for (x,y,i) in zip(xᵢ,yᵢ,2:length(xᵢ))
        k₁ = h*f(x,y)
        k₂ = h*f(x+h/2,y+k₁/2)
        k₃ = h*f(x+h/2,y+k₂/2)
        k₄ = h*f(x+h,y+k₃)

        yᵢ[i] = y + (k₁+2k₂+2k₃+k₄)/6
    end
    
    return yᵢ
end

function Adams_Multon(f::Function,xlim,yₒ,h)
    xᵢ = xlim[1]:h:xlim[2]
    y₁₋₄ = RK4(f,[xᵢ[1],xᵢ[4]],y₀,h)
    
    yᵢ = [y₁₋₄ ;zeros(length(xᵢ)-4)]

    for i in 5:length(xᵢ)
        yᵢ[i] = yᵢ[i-1] + h/24*( 55f(xᵢ[i-1],yᵢ[i-1]) - 59f(xᵢ[i-2],yᵢ[i-2]) + 37f(xᵢ[i-3],yᵢ[i-3]) - 9f(xᵢ[i-4],yᵢ[i-4]))
    end
    
    return yᵢ
end

function Adams_Multon_Corrector(f::Function,xlim,y₀,h)
    xᵢ = xlim[1]:h:xlim[2]

    yᵢ = Adams_Multon(f,xlim,y₀,h)

    for i in 4:length(xᵢ)
        yᵢ[i] = yᵢ[i-1] + h/24*( 9f(xᵢ[i],yᵢ[i]) + 19f(xᵢ[i-1],yᵢ[i-1]) - 5f(xᵢ[i-2],yᵢ[i-2]) + f(xᵢ[i-3],yᵢ[i-3]) )
    end 

    return yᵢ
end

Adams_Multon_Corrector (generic function with 1 method)

### Testing and Comparisons
The function $f(x,y) = \frac{dy}{dx}$ and its analytical solution $y(x)$ is defined below. An initial condition is set

In [3]:
function f(x,y)
    y + x^2 - 2x +sin(x)
end

y₀ = 0.1

function y(x)
    0.6exp(x) - x^2 -1/2*(cos(x) + sin(x))
end

y (generic function with 1 method)

In [27]:
function ODE_Test(f,y,lim,yₒ,N_lim,x_eval;plot="N",scale = "loglog")
    true_y = y(x_eval)
    if scale == "loglog"
        pl_func = loglog
    else
        pl_func = plt.plot
    end

    plt.close("all")
    
    for (ODE_Func,FE) in zip([RK4,Adams_Multon,Adams_Multon_Corrector],[4,1,2])
        N_arr = round.(Int,exp(1) .^(range(log(N_lim[1]),stop=log(N_lim[2]),length=100)))
        Err_arr=zeros(length(N_arr))

        for (n,i) in zip(N_arr,1:length(N_arr))
            h = (lim[2]-lim[1])/n
            est_y = ODE_Func(f,lim,yₒ,h)
            Err = abs(true_y - est_y[end])
        
            Err_arr[i] = Err
        end
        if plot == "N"
            pl_func(N_arr,Err_arr)
        elseif plot == "FE"
            pl_func(FE*N_arr,Err_arr)
        end
    end

    legend(string.(["RK4","Adams Multon","Adams Multon Corrector"]))
    if plot == "N"
        xlabel("N")
    elseif plot == "FE"
        xlabel("Function Evaluations")
    end
    ylabel("Deviation from True Value")
    title("Error for "*string(f))
    grid()
    plt.show()
end


ODE_Test (generic function with 1 method)

#### Plotting absolute error as a function of number of steps taken

In [29]:
ODE_Test(f,y,[0,2],y₀,[10,5000],2,plot="N",scale="loglog")


![alt text](Fig_1.png) <br>
We see that The error for all decreases linearly and is lowest for RK4, then Adams Multon with corrector and highest for Adams Multon

### Non Ideal Air Resistance
We try to solve the Bernoulli type differential equation numerically:<br>
$v^\prime(t) + \mu (t) v(t) -\omega^2 (t)v^3(t) =0$

With initial condition that $v_0 = 2$

The true solution is (I think): <br>
1. $v(t)=\sqrt{μ(t)} * [ω^2(t) + e^{2μ(t)t + 2ln\sqrt{-μ(t)/4+ω(t)^2}}]^{-1/2}$ for $μ(t)-4ω(t)^2<0$
2. $v(t)=\sqrt{μ(t)} * [ω^2(t) + e^{2μ(t)t + ln\{μ(t)/4-ω(t)^2\}}]^{-1/2}$ for $μ(t)-4ω(t)^2>0$

In [ ]:
function v′(t,μ,v,ω)
    ω(t)^2*v^3 - μ(t)*v    
end

v₀=2

function v(t,μ,ω)
    if μ(t)-4ω(t)^2<0
        sqrt(μ(t)) * (ω(t)^2 + exp( 2μ(t)t +2log2(sqrt(-(μ(t)/4-ω(t)^2)))))^(-1/2)
    else

        sqrt(μ(t)) * (ω(t)^2 + exp( 2μ(t)t +log2(-ω(t)^2+μ(t)/4)))^(-1/2)
    end
end

function v_const_2(t)
    2exp((0.12496225t^4-2t))
end

v_const3 (generic function with 1 method)

#### Constant $\omega$ and $\mu$

In [40]:
function ω₀(t)
    0.707
end

function μ₀(t)
    2
end
t_test=1.5

v′_const= (t,v)->v′(t,μ₀,v,ω₀)
v_const= t->v(t,μ₀,ω₀)

#34 (generic function with 1 method)

I'm a bit unsure about this one, the solution on the sheet wasn't correct so I tried changing the signs such that it made sense, but the two algorithms converge to different values and apparently RK4 is more accurate.

In [50]:
ODE_Test(v′_const,v_const,[0,1.5],v₀,[10,5000],1.5,plot="N",scale="loglog")

![alt text](Fig_2.png)

Something must be going wrong with the code below, might look at it later

In [51]:
# ω₁ = 1/sqrt(2)-0.2:0.01:1/sqrt(2)+0.2
function ωₜ(t;t_max=1.5)
    ω₁ = 1/sqrt(2)-0.2:0.01:1/sqrt(2)+0.2
    return ω₁[(1+round(Int,t/t_max*(length(ω₁)-1)))]
end

ωₜ (generic function with 1 method)

In [52]:
t=0:0.05:1.5
plot(t,v.(t,μ₀,ωₜ))
show()

In [55]:
t=0:0.05:1.5
plot(t,v.(t,μ₀,ωₜ))
plot(t,Adams_Multon_Corrector((t,v)->v′(t,μ₀,v,ωₜ) ,[t[1],t[end]],y₀,0.05))
plot(t,RK4((t,v)->v′(t,μ₀,v,ωₜ) ,[t[1],t[end]],y₀,0.05))
show()

In [54]:
function ω₂(t)
    ω₀ = 1/sqrt(2)
    ω₀*tan(t)
end

ω₂ (generic function with 1 method)

In [58]:
t=0:0.05:1.5
plot(t,v.(t,μ₀,ωₜ))
plot(t,Adams_Multon_Corrector((t,v)->v′(t,μ₀,v,ω₂) ,[t[1],t[end]],y₀,0.05))
plot(t,RK4((t,v)->v′(t,μ₀,v,ω₂) ,[t[1],t[end]],y₀,0.05))
show()